## Need to upload the following files before launching this notebook

- database.zip (spider dataset sqlite databases)
- train.jsonl
- val.jsonl
- test.jsonl

In [ ]:
!pip install -q transformers datasets evaluate mlflow accelerate sentencepiece


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 32.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.2/28.2 MB 92.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 116.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.9/231.9 kB 24.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from transformers import (
    AutoTokenizer, AutoModelForSeq2SeqLM,
    Seq2SeqTrainer, Seq2SeqTrainingArguments,
    DataCollatorForSeq2Seq
)
from datasets import load_dataset
from huggingface_hub import login
import mlflow
import os
from evaluate import load
import tqdm
import re
import sqlite3
import sqlparse


In [ ]:
import mlflow
mlflow.set_tracking_uri("file:/content/mlruns")

mlflow.set_experiment("codet5p_spider_finetune")


2025/04/23 16:17:03 INFO mlflow.tracking.fluent: Experiment with name 'codet5p_spider_finetune' does not exist. Creating a new experiment.


<Experiment: artifact_location='file:///content/mlruns/617251086665708557', creation_time=1745425023866, experiment_id='617251086665708557', last_update_time=1745425023866, lifecycle_stage='active', name='codet5p_spider_finetune', tags={}>

In [ ]:

from google.colab import userdata
HF_TOKEN = userdata.get("HF_TOKEN")
login(HF_TOKEN)

## Model Loading

In [ ]:
model_name = "Salesforce/codet5p-770m"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)


tokenizer_config.json:   0%|          | 0.00/1.48k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/703k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/294k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/12.5k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/770 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.48G [00:00<?, ?B/s]

## Dataset Preparation

In [ ]:
data_files = {
    "train": "/content/train.jsonl",
    "validation": "/content/val.jsonl",
    "test": "/content/test.jsonl"
}
dataset = load_dataset("json", data_files=data_files)


Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [ ]:
def preprocess(example):
    inputs = tokenizer(
        example["prompt"],
        truncation=True,
        padding="max_length",
        max_length=768
    )
    targets = tokenizer(
        example["completion"],
        truncation=True,
        padding="max_length",
        max_length=256
    )
    inputs["labels"] = targets["input_ids"]
    return inputs


tokenized = dataset.map(preprocess, batched=True, remove_columns=dataset["train"].column_names)


Map:   0%|          | 0/6300 [00:00<?, ? examples/s]

Map:   0%|          | 0/350 [00:00<?, ? examples/s]

Map:   0%|          | 0/350 [00:00<?, ? examples/s]

## Evaluation Metrics

In [ ]:
VALID_DB_IDS = [row["db_id"] for row in dataset["validation"]]

In [ ]:
!unzip -q /content/database.zip -d /content/


In [ ]:
def validate_query(sql):
    if re.search(r'\bJOIN\b.*?(?=\bWHERE\b|\bGROUP\b|\bORDER\b|;|$)', sql, re.IGNORECASE):
        if not re.search(r'\bON\b', sql, re.IGNORECASE):
            # print(f"Invalid query: Missing ON clause for JOIN: {sql}")
            return None
    parsed = sqlparse.parse(sql)
    if not parsed or not parsed[0].tokens:
        # print(f"Invalid query syntax: {sql}")
        return None
    return True


def run_sql_query(db_path, sql):
    try:
        conn = sqlite3.connect(db_path)
        cursor = conn.cursor()
        # print(f"Running query on {db_path}:\n{sql}")
        cursor.execute(sql)
        result = cursor.fetchall()
        conn.close()
        return result
    except Exception as e:
        # print(f"Error running query on {db_path}:\n{sql}\n{e}")
        return None



def compute_execution_match(preds, references, db_ids, db_dir="/content/database"):
    correct = 0
    both_none = 0
    total = len(preds)
    invalid_queries = 0

    for pred, ref, db in tqdm.tqdm(zip(preds, references, db_ids), total=total, desc="Evaluating Execution Match"):
        db_path = f"{db_dir}/{db}/{db}.sqlite"
        if validate_query(pred):
            pred_result = run_sql_query(db_path, pred)
            ref_result = run_sql_query(db_path, ref)

            if pred_result is None and ref_result is None:
                both_none += 1
            elif pred_result == ref_result:
                correct += 1
        else:
            invalid_queries += 1

    return correct / total, invalid_queries/total

In [ ]:
exact_match = load("exact_match")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    metrics = exact_match.compute(predictions=preds, references=labels)
    exec_match, invalid_ratio = compute_execution_match(
        preds,
        labels,
        VALID_DB_IDS,
        db_dir="/content/database"
    )
    metrics["eval_execution_match"] = exec_match
    metrics["invalid_sql"] = invalid_ratio

    return metrics


## Model Finetuning

In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir="./codet5p_sql_finetuned",
    eval_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=1,
    load_best_model_at_end=True,
    learning_rate=5e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=5,
    weight_decay=0.01,
    predict_with_generate=True,
    logging_dir="./logs",
    logging_steps=50,
    report_to="mlflow",
    metric_for_best_model="eval_exact_match",
    greater_is_better=True,
    gradient_checkpointing=True,
)


In [ ]:
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized["train"],
    eval_dataset=tokenized["validation"],
    tokenizer=tokenizer,
    data_collator=DataCollatorForSeq2Seq(tokenizer, model=model),
    compute_metrics=compute_metrics,
)


<ipython-input-55-f06de53d0e6e>:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [ ]:
with mlflow.start_run():
    mlflow.log_param("model", model_name)
    mlflow.log_params(training_args.to_dict())

    trainer.train()
    VALID_DB_IDS = [row["db_id"] for row in dataset["test"]]
    results = trainer.evaluate(tokenized["test"])

    for k, v in results.items():
        mlflow.log_metric("test_"+k, v)

    print(results)



Epoch,Training Loss,Validation Loss,Execution Match,Exact Match,Invalid Sql
1,0.013900,0.018471,0.240000,0.205714,0.154286
2,0.008500,0.016957,0.248571,0.220000,0.140000
3,0.006600,0.015505,0.248571,0.228571,0.180000
4,0.005300,0.015258,0.254286,0.234286,0.177143
5,0.005200,0.014858,0.254286,0.234286,0.217143


Evaluating Execution Match: 100%|██████████| 350/350 [00:00<00:00, 1075.09it/s]
There were missing keys in the checkpoint model loaded: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight', 'lm_head.weight'].


Evaluating Execution Match: 100%|██████████| 350/350 [00:00<00:00, 887.56it/s]


{'eval_execution_match': 0.27714285714285714, 'eval_loss': 0.0198301300406456, 'eval_exact_match': 0.25142857142857145, 'eval_invalid_sql': 0.17714285714285713, 'eval_runtime': 68.0649, 'eval_samples_per_second': 5.142, 'eval_steps_per_second': 0.646, 'epoch': 5.0}


## Push Model to HuggingFace

In [ ]:
trainer.push_to_hub("rajiv8197/finetuned-codet5p-sql")

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

model.safetensors:   0%|          | 0.00/2.95G [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/5.43k [00:00<?, ?B/s]

## Download MLFlow logs

In [ ]:
!zip -r mlruns_with_eval.zip mlruns

  adding: mlruns/ (stored 0%)
  adding: mlruns/617251086665708557/ (stored 0%)
  adding: mlruns/617251086665708557/3344665580e846b7a9fb41117705e80d/ (stored 0%)
  adding: mlruns/617251086665708557/3344665580e846b7a9fb41117705e80d/metrics/ (stored 0%)
  adding: mlruns/617251086665708557/3344665580e846b7a9fb41117705e80d/metrics/learning_rate (deflated 58%)
  adding: mlruns/617251086665708557/3344665580e846b7a9fb41117705e80d/metrics/loss (deflated 60%)
  adding: mlruns/617251086665708557/3344665580e846b7a9fb41117705e80d/metrics/grad_norm (deflated 55%)
  adding: mlruns/617251086665708557/3344665580e846b7a9fb41117705e80d/metrics/epoch (deflated 61%)
  adding: mlruns/617251086665708557/3344665580e846b7a9fb41117705e80d/params/ (stored 0%)
  adding: mlruns/617251086665708557/3344665580e846b7a9fb41117705e80d/params/fp16_opt_level (stored 0%)
  adding: mlruns/617251086665708557/3344665580e846b7a9fb41117705e80d/params/do_predict (stored 0%)
  adding: mlruns/617251086665708557/3344665580e846b7a9f

In [ ]:
from google.colab import files
files.download("mlruns.zip")


FileNotFoundError: Cannot find file: mlruns.zip

## Exporting Predictions

In [ ]:
predictions = trainer.predict(tokenized["test"])
decoded_preds = tokenizer.batch_decode(predictions.predictions, skip_special_tokens=True)
decoded_labels = tokenizer.batch_decode(tokenized["test"]["labels"], skip_special_tokens=True)
db_ids = [x["db_id"] for x in dataset["test"]]
prompts = [x["prompt"] for x in dataset["test"]]


In [ ]:
import json

with open("predictions.jsonl", "w") as f:
    for db, prompt, pred, label in zip(db_ids, prompts, decoded_preds, decoded_labels):
        json.dump({
            "db_id": db,
            "prompt": prompt,
            "gold_sql": label,
            "predicted_sql": pred
        }, f)
        f.write("\n")


In [ ]:
files.download("predictions-aftereval.jsonl")